# Notebook 04: MCP (Model Context Protocol) Tools

## 🎯 What is This Notebook About?

Welcome to Notebook 04! In this notebook, we'll explore **MCP (Model Context Protocol)** - a protocol for integrating external tools and services with LLMs.

**What we'll learn:**
1. **What is MCP** - Understanding the Model Context Protocol
2. **Creating Agents with MCP Tools** - How to configure agents with MongoDB MCP tools
3. **Agent Inference** - How to query agents and see them use tools
4. **Understanding Tool Execution** - How agents call tools and use results

**Why this matters:**
- LLMs can't directly interact with systems
- MCP provides a standardized way to connect tools
- Enables agents to take real actions
- Makes agents more powerful and useful

---

## 📚 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand what MCP is and why it's important
- ✅ Know how to create agents with MCP toolgroups
- ✅ Learn how to query agents and retrieve responses
- ✅ See how agents automatically use tools when needed
- ✅ Understand the agent inference workflow

---

## ⚙️ Prerequisites

- LlamaStack server running on OpenShift (see Module README)
- MongoDB MCP server deployed and registered (see OpenShift docs)
- Python environment with dependencies installed
- Understanding of Notebook 03 (LlamaStack Core Features)

---

## 🔧 Setup

Let's start by connecting to LlamaStack and verifying everything is working.


In [ ]:
# Import required libraries
import os
import subprocess
import time
import httpx
import urllib3
from llama_stack_client import LlamaStackClient

# Suppress SSL warnings for OpenShift self-signed certificates
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ============================================================================
# Configuration - Update these values for your OpenShift deployment
# ============================================================================

def get_llamastack_url() -> str:
    """Get LlamaStack URL from environment or OpenShift route"""
    # Check environment variable first
    url = os.getenv("LLAMA_STACK_URL")
    if url:
        return url.rstrip("/")
    
    # Try to get from OpenShift route
    namespace = os.getenv("NAMESPACE", "my-first-model")
    
    try:
        result = subprocess.run(
            ["oc", "get", "route", "llamastack-route", "-n", namespace,
             "-o", "jsonpath={.spec.host}"],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0 and result.stdout:
            return f"https://{result.stdout.strip()}"
    except Exception:
        pass
    
    # Fallback to localhost
    return "http://localhost:8321"

# Get LlamaStack URL
llamastack_url = get_llamastack_url()

# Model identifier - Use the full identifier from LlamaStack
model = os.getenv("LLAMA_MODEL", "vllm-inference/llama-32-3b-instruct")

print(f"📡 LlamaStack URL: {llamastack_url}")
print(f"🤖 Model: {model}")

# Initialize LlamaStack client
client = LlamaStackClient(base_url=llamastack_url)

# Verify connection
try:
    models = client.models.list()
    model_count = len(models.data) if hasattr(models, 'data') else len(models)
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {model_count}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Check if route exists: oc get route llamastack-route -n my-first-model")
    print("   2. Update llamastack_url variable above with your route URL")
    print("   3. Or set LLAMA_STACK_URL environment variable:")
    print("      export LLAMA_STACK_URL='https://<route-host>'")
    raise


---

## Part 1: Understanding MCP (Model Context Protocol)

**What we're doing:** Understanding MCP - the protocol that lets agents interact with external systems.

**Why:** LLMs are just text processors - they can't directly interact with systems. MCP provides a standardized way to connect tools, so agents can actually DO things, not just talk about them!

### What is MCP?

**MCP (Model Context Protocol)** is a protocol for integrating external tools and services with LLMs. It allows agents to:
- **Call external APIs** (e.g., check service status, restart services)
- **Access databases** (e.g., query MongoDB collections)
- **Execute commands** (e.g., run system commands)
- **Integrate with other systems** (e.g., monitoring tools, ticketing systems)

**Why MCP matters:**
- LLMs can't directly interact with systems (they're text processors, not system operators!)
- MCP provides a standardized way to connect tools (one protocol, many tools)
- Enables agents to take real actions (check status, query databases, execute commands)
- Makes agents more powerful and useful (from assistant to operator!)

**MCP Architecture:**
- **MCP Server**: Exposes tools, resources, and prompts (e.g., MongoDB MCP server)
- **MCP Client (LlamaStack)**: Connects to MCP servers and makes tools available to agents
- **Agent**: Uses tools exposed by MCP servers to take actions

**In this notebook:** We'll use the MongoDB MCP server that's already deployed in OpenShift. The agent will automatically use MongoDB tools to query the database when needed!


---

## Part 2: Creating an Agent with MCP Tools

**What we're doing:** Creating an agent that has access to MongoDB MCP tools.

**Why:** Agents need to be configured with toolgroups to access MCP tools. Once configured, the agent can automatically decide when to use tools based on the user's query.

**Key Points:**
- `toolgroups`: List of MCP toolgroups the agent can use (e.g., `["mcp::mongodb"]`)
- `tool_choice`: Set to `"auto"` to let the agent decide when to use tools
- `instructions`: Guide the agent on when and how to use tools
- `sampling_params`: Control response generation (max_tokens, temperature, etc.)

**The fun part:** Once created, the agent will automatically use MongoDB tools when you ask database-related questions!


In [ ]:
# Create an agent with MongoDB MCP tools
print("=" * 60)
print("Creating Agent with MongoDB MCP Tools")
print("=" * 60)

agent_config = {
    "model": model,
    "instructions": (
        "You have access to MongoDB through MCP tools. "
        "Always use the MongoDB MCP tools to query the database when asked about databases or data."
    ),
    "toolgroups": ["mcp::mongodb"],  # Include MongoDB MCP toolgroup
    "tool_choice": "auto",  # Let agent decide when to use tools
    "sampling_params": {
        "max_tokens": 2000  # Must be > 0!
    }
}

print("\n📋 Agent Configuration:")
print(f"   Model: {agent_config['model']}")
print(f"   Toolgroups: {agent_config['toolgroups']}")
print(f"   Tool Choice: {agent_config['tool_choice']}")
print(f"   Max Tokens: {agent_config['sampling_params']['max_tokens']}")

# Create agent using alpha API
try:
    agent = client.alpha.agents.create(agent_config=agent_config)
    print(f"\n✅ Agent created successfully!")
    print(f"   Agent ID: {agent.agent_id}")
    print(f"\n💡 This agent can now use MongoDB MCP tools!")
except Exception as e:
    print(f"\n❌ Error creating agent: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Make sure MongoDB MCP toolgroup is registered:")
    print("      Check: oc get toolgroups -n my-first-model")
    print("   2. Verify MongoDB MCP server is running:")
    print("      Check: oc get pods -n my-first-model | grep mongodb-mcp")
    print("   3. Register toolgroup if needed (see OpenShift docs)")
    raise


---

## Part 3: Creating a Session

**What we're doing:** Creating a session for the agent to maintain conversation context.

**Why:** Sessions allow agents to maintain context across multiple interactions. Each session is independent, so you can have multiple conversations with the same agent.

**Key Points:**
- Sessions maintain conversation history
- Each session is independent (separate conversations)
- Session names are optional but helpful for organization
- Sessions persist until explicitly deleted


In [ ]:
# Create a session for the agent
print("=" * 60)
print("Creating Session")
print("=" * 60)

try:
    session = client.alpha.agents.session.create(
        agent_id=agent.agent_id,
        session_name="mcp-tools-demo"
    )
    print(f"\n✅ Session created successfully!")
    print(f"   Session ID: {session.session_id}")
    print(f"   Session Name: {session.session_name if hasattr(session, 'session_name') else 'N/A'}")
except Exception as e:
    print(f"\n❌ Error creating session: {e}")
    raise


---

## Part 4: Querying the Agent

**What we're doing:** Querying the agent and watching it use MongoDB MCP tools automatically.

**Why:** This demonstrates the power of MCP - the agent automatically decides when to use tools based on your query. You don't need to explicitly tell it to use tools!

**The Workflow:**
1. **Send query** to the agent
2. **Agent reasons** about the query
3. **Agent decides** to use MongoDB tools (automatically!)
4. **Agent calls** MongoDB MCP tools
5. **Agent receives** tool results
6. **Agent generates** response using real data

**The fun part:** Watch the agent think and act - it will automatically use MongoDB tools when you ask database questions!


In [ ]:
# Helper function to convert objects to dictionaries
def to_dict(obj):
    """Convert object to dictionary"""
    if isinstance(obj, dict):
        return obj
    if hasattr(obj, 'model_dump'):
        return obj.model_dump()
    if hasattr(obj, 'dict'):
        return obj.dict()
    if hasattr(obj, '__dict__'):
        return {k: v for k, v in obj.__dict__.items() if not k.startswith('_')}
    return {}

# Query the agent
print("=" * 60)
print("Querying Agent")
print("=" * 60)

query = "What collections are in the mcp_demo database?"
print(f"\n👤 Query: {query}\n")

# Create turn with streaming
try:
    turn_stream = client.alpha.agents.turn.create(
        agent_id=agent.agent_id,
        session_id=session.session_id,
        messages=[{"role": "user", "content": query}],
        stream=True
    )
    print("✅ Turn created, streaming response...")
except Exception as e:
    print(f"\n❌ Error creating turn: {e}")
    raise

# Extract turn_id from streaming response
print("\n📋 Extracting turn_id from stream...")
turn_id = None
for chunk in turn_stream:
    if hasattr(chunk, 'event') and chunk.event and hasattr(chunk.event, 'payload'):
        payload = chunk.event.payload
        d = to_dict(payload)
        turn_id = d.get('turn_id')
        if not turn_id and 'step_details' in d:
            step_details = to_dict(d['step_details'])
            turn_id = step_details.get('turn_id')
        if turn_id:
            break

if turn_id:
    print(f"✅ Turn ID: {turn_id}")
else:
    print("⚠️  Could not extract turn_id from stream")


---

## Part 5: Retrieving Turn Results

**What we're doing:** Retrieving the complete turn results to see the agent's response and tool usage.

**Why:** The streaming response gives us the turn_id, but we need to retrieve the full turn to see:
- The agent's final response
- Which tools were called
- Tool arguments and results
- Step-by-step execution details

**Key Points:**
- Wait a moment after streaming (turn needs time to complete)
- Retrieve turn using the turn_id
- Parse messages and steps to extract response
- Check steps to see tool calls and results


In [ ]:
# Retrieve turn results
import json

if turn_id:
    print("=" * 60)
    print("Retrieving Turn Results")
    print("=" * 60)
    
    print("\n⏳ Waiting for turn to complete...")
    time.sleep(2)  # Give the turn time to complete
    
    try:
        # Retrieve the turn
        response = httpx.get(
            f"{llamastack_url}/v1alpha/agents/{agent.agent_id}/session/{session.session_id}/turn/{turn_id}",
            verify=False,
            timeout=30
        )
        response.raise_for_status()
        
        # Handle null response
        if not response.text or response.text.strip() == 'null':
            print("   ⚠️  Turn not ready yet (response was null)")
            print("   💡 The turn may still be processing. Try waiting a bit longer.")
            data = None
        else:
            data = response.json()
        
        if data is None:
            print("   ⚠️  Could not retrieve turn data")
            print("   💡 This might mean the turn is still processing or the turn_id is incorrect")
        else:
            print("✅ Turn data retrieved successfully!")
            
            # Extract messages and steps
            messages = data.get('messages', [])
            steps = data.get('steps', [])
            
            print(f"\n📊 Turn Summary:")
            print(f"   Messages: {len(messages)}")
            print(f"   Steps: {len(steps)}")
            
            # Debug: Print all steps for full visibility
            if steps:
                print(f"\n{'='*60}")
                print("📦 Full Steps Debug (All Step Objects)")
                print(f"{'='*60}\n")
                
                for i, step in enumerate(steps):
                    step_type = step.get('type', '') or step.get('step_type', '')
                    print(f"{'='*60}")
                    print(f"Step {i}: type='{step_type}'")
                    print(f"{'='*60}")
                    print(json.dumps(step, indent=2, default=str))
                    print()
            else:
                print("\n⚠️  No steps found in turn data")
                print("\n📦 Full Data Object (for debugging):")
                data_str = json.dumps(data, indent=2, default=str)
                if len(data_str) > 2000:
                    print(data_str[:2000] + "...")
                else:
                    print(data_str)
            
    except Exception as e:
        print(f"\n❌ Error retrieving turn: {e}")
        import traceback
        traceback.print_exc()
        data = None
else:
    print("\n⚠️  Cannot retrieve turn results without turn_id")
    data = None


---

## Part 6: Analyzing Tool Usage

**What we're doing:** Examining the turn steps to see which tools were called and how they were used.

**Why:** Understanding tool usage helps you:
- See what the agent decided to do
- Verify tools were called correctly
- Debug issues with tool execution
- Understand the agent's reasoning process

**What to look for:**
- **tool_call** steps: Show which tools were called and with what arguments
- **tool_result** steps: Show the results returned by tools
- **inference** steps: Show the agent's reasoning and final response


In [ ]:
# Analyze tool usage from steps (tool-specific only)
import json

if data:
    print("=" * 60)
    print("Analyzing Tool Usage")
    print("=" * 60)
    
    steps = data.get('steps', [])
    
    if steps:
        # Filter and show only tool-related steps
        tool_steps = []
        for i, step in enumerate(steps):
            step_type = step.get('type', '') or step.get('step_type', '')
            
            # Check for tool calls in inference steps
            if step_type == 'inference':
                model_response = step.get('model_response', {})
                tool_calls = model_response.get('tool_calls', [])
                if tool_calls:
                    tool_steps.append((i, step, 'tool_call', tool_calls))
            
            # Check for tool execution steps
            if step_type == 'tool_execution':
                tool_execution = step.get('tool_execution', {})
                if tool_execution:
                    tool_steps.append((i, step, 'tool_execution', tool_execution))
            
            # Check for tool result steps
            if step_type == 'tool_result':
                tool_result = step.get('tool_result', {})
                if tool_result:
                    tool_steps.append((i, step, 'tool_result', tool_result))
        
        if tool_steps:
            print(f"\n🔧 Found {len(tool_steps)} tool-related step(s):\n")
            
            for step_idx, step, tool_type, tool_data in tool_steps:
                print(f"{'='*60}")
                print(f"Step {step_idx}: {tool_type}")
                print(f"{'='*60}")
                
                if tool_type == 'tool_call':
                    print("\n📋 Tool Calls from Inference Step:")
                    for tool_call in tool_data:
                        call_id = tool_call.get('call_id', 'unknown')
                        tool_name = tool_call.get('tool_name', 'unknown')
                        arguments = tool_call.get('arguments', '{}')
                        print(f"\n  🔧 Tool: {tool_name}")
                        print(f"     Call ID: {call_id}")
                        try:
                            args_dict = json.loads(arguments) if isinstance(arguments, str) else arguments
                            print(f"     Arguments: {json.dumps(args_dict, indent=6)}")
                        except:
                            print(f"     Arguments: {arguments}")
                
                elif tool_type == 'tool_execution':
                    print("\n⚙️  Tool Execution:")
                    tool_name = tool_data.get('tool_name', 'unknown')
                    call_id = tool_data.get('call_id', 'unknown')
                    print(f"  Tool: {tool_name}")
                    print(f"  Call ID: {call_id}")
                    if 'arguments' in tool_data:
                        args = tool_data.get('arguments', {})
                        print(f"  Arguments: {json.dumps(args, indent=4, default=str)}")
                
                elif tool_type == 'tool_result':
                    print("\n📤 Tool Result:")
                    tool_name = tool_data.get('name', tool_data.get('tool_name', 'unknown'))
                    result_content = tool_data.get('content', '')
                    print(f"  Tool: {tool_name}")
                    if result_content:
                        result_str = str(result_content)
                        if len(result_str) > 500:
                            print(f"  Result: {result_str[:500]}...")
                            print(f"  (Full result: {len(result_str)} characters)")
                        else:
                            print(f"  Result: {result_str}")
                
                print()
        else:
            print("\n⚠️  No tool-related steps found")
            print("   💡 The agent may not have used any tools, or tool information is in a different format")
            print("\n   Available step types:")
            for i, step in enumerate(steps):
                step_type = step.get('type', '') or step.get('step_type', '')
                print(f"     Step {i}: {step_type}")
    else:
        print("\n⚠️  No steps found in turn data")
else:
    print("\n⚠️  No turn data available")
    print("   💡 Make sure Part 5 completed successfully")


---

## Part 7: Extracting the Agent's Response

**What we're doing:** Extracting the final response text from the turn data.

**Why:** The agent's response might be in different places:
- In the `messages` array (assistant messages)
- In the `steps` array (inference steps with model_response)
- We need to check both to find the complete response

**Key Points:**
- Check messages first (most common location)
- Fall back to steps if not found in messages
- Handle different content formats (string, list, dict)


In [ ]:
# Extract the agent's response
if data:
    print("=" * 60)
    print("Extracting Agent Response")
    print("=" * 60)
    
    messages = data.get('messages', [])
    steps = data.get('steps', [])
    response_text = None
    
    # Try messages first
    print("\n🔍 Checking messages...")
    for msg in reversed(messages):
        if msg.get('role') == 'assistant':
            content = msg.get('content', '')
            if isinstance(content, str) and content.strip():
                response_text = content
                print("   ✅ Found response in messages")
                break
            elif isinstance(content, list) and content:
                for item in content:
                    if isinstance(item, dict):
                        text = item.get('text', '')
                        if text:
                            response_text = text
                            print("   ✅ Found response in messages (list format)")
                            break
                    elif isinstance(item, str) and item.strip():
                        response_text = item
                        print("   ✅ Found response in messages (list format)")
                        break
                if response_text:
                    break
    
    # If not found in messages, check steps
    if not response_text:
        print("   ⚠️  Not found in messages, checking steps...")
        for step in reversed(steps):
            # Try inference step
            if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    print("   ✅ Found response in inference step")
                    break
            
            # Try any step with model_response
            if 'model_response' in step:
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    print("   ✅ Found response in step model_response")
                    break
    
    # Display the response
    print("\n" + "=" * 60)
    print("Agent Response")
    print("=" * 60)
    
    if response_text:
        print(f"\n💬 {response_text}\n")
        
        # Show tools used
        tools_used = []
        for step in steps:
            if step.get('type') == 'tool_call' or step.get('step_type') == 'tool_call':
                tool_call = step.get('tool_call', {})
                tool_name = tool_call.get('name', 'unknown')
                if tool_name not in tools_used:
                    tools_used.append(tool_name)
        
        if tools_used:
            print(f"🔧 Tools used: {', '.join(tools_used)}\n")
        else:
            print("💡 No tools were used in this response\n")
    else:
        print("\n⚠️  Could not extract response text")
        print("   💡 The turn may still be processing, or response format is unexpected")
        print("   💡 Check the steps above for more details")


---

## Part 8: Testing Multiple Queries

**What we're doing:** Testing the agent with different queries to see how it uses MongoDB tools.

**Why:** Different queries will trigger different tool calls. This helps you understand:
- When the agent decides to use tools
- Which tools are called for different queries
- How the agent combines tool results into responses

**Try different queries:**
- "What databases are available?"
- "How many documents are in the incidents collection?"
- "What are the fields in the incidents collection?"


In [ ]:
# Test with multiple queries
import json

def extract_tools_from_steps(steps):
    """Extract tool names from steps (checking both tool_execution and inference steps)"""
    tools_used = []
    for step in steps:
        step_type = step.get('type', '') or step.get('step_type', '')
        
        # Check tool_execution steps
        if step_type == 'tool_execution':
            tool_execution = step.get('tool_execution', {})
            tool_name = tool_execution.get('tool_name', '')
            if tool_name and tool_name not in tools_used:
                tools_used.append(tool_name)
        
        # Check inference steps for tool_calls
        elif step_type == 'inference':
            model_response = step.get('model_response', {})
            tool_calls = model_response.get('tool_calls', [])
            for tool_call in tool_calls:
                tool_name = tool_call.get('tool_name', '')
                if tool_name and tool_name not in tools_used:
                    tools_used.append(tool_name)
    
    return tools_used

def extract_response_text(data):
    """Extract response text from messages or steps"""
    messages = data.get('messages', [])
    steps = data.get('steps', [])
    response_text = None
    
    # Try messages first
    for msg in reversed(messages):
        if msg.get('role') == 'assistant':
            content = msg.get('content', '')
            if isinstance(content, str) and content.strip():
                response_text = content
                break
            elif isinstance(content, list) and content:
                for item in content:
                    if isinstance(item, dict):
                        text = item.get('text', '')
                        if text:
                            response_text = text
                            break
                    elif isinstance(item, str) and item.strip():
                        response_text = item
                        break
                if response_text:
                    break
    
    # If not found in messages, check steps
    if not response_text:
        for step in reversed(steps):
            if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    break
    
    return response_text

def retrieve_turn_with_retry(llamastack_url, agent_id, session_id, turn_id, max_retries=5, initial_wait=2):
    """Retrieve turn with retry logic"""
    for attempt in range(max_retries):
        wait_time = initial_wait * (2 ** attempt)  # Exponential backoff
        if attempt > 0:
            print(f"   ⏳ Retry {attempt}/{max_retries-1} (waiting {wait_time}s)...")
            time.sleep(wait_time)
        else:
            time.sleep(wait_time)
        
        try:
            response = httpx.get(
                f"{llamastack_url}/v1alpha/agents/{agent_id}/session/{session_id}/turn/{turn_id}",
                verify=False,
                timeout=30
            )
            response.raise_for_status()
            
            if response.text and response.text.strip() != 'null':
                data = response.json()
                steps = data.get('steps', [])
                
                # Check if turn is complete (has final inference step with content)
                is_complete = False
                for step in reversed(steps):
                    if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                        model_response = step.get('model_response', {})
                        content = model_response.get('content', '')
                        if content and content.strip():
                            is_complete = True
                            break
                
                if is_complete or len(steps) > 0:
                    return data
                elif attempt < max_retries - 1:
                    continue  # Try again
            
        except Exception as e:
            if attempt < max_retries - 1:
                continue
            else:
                raise
    
    return None

print("=" * 60)
print("Testing Multiple Queries")
print("=" * 60)

test_queries = [
    "What collections are in the mcp_demo database?",
    "How many documents are in the mcp_demo.incidents collection?",
    "Show me one document from the mcp_demo.incidents collection",
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"Test {i}: {query}")
    print('='*60)
    
    try:
        # Create turn
        print("   📤 Creating turn...")
        turn_stream = client.alpha.agents.turn.create(
            agent_id=agent.agent_id,
            session_id=session.session_id,
            messages=[{"role": "user", "content": query}],
            stream=True
        )
        
        # Get turn_id
        print("   🔍 Extracting turn_id...")
        turn_id = None
        for chunk in turn_stream:
            if hasattr(chunk, 'event') and chunk.event and hasattr(chunk.event, 'payload'):
                payload = chunk.event.payload
                d = to_dict(payload)
                turn_id = d.get('turn_id')
                if not turn_id and 'step_details' in d:
                    step_details = to_dict(d['step_details'])
                    turn_id = step_details.get('turn_id')
                if turn_id:
                    break
        
        if turn_id:
            print(f"   ✅ Turn ID: {turn_id}")
            print("   ⏳ Retrieving turn results...")
            
            # Retrieve with retry
            data = retrieve_turn_with_retry(
                llamastack_url, 
                agent.agent_id, 
                session.session_id, 
                turn_id
            )
            
            if data:
                steps = data.get('steps', [])
                messages = data.get('messages', [])
                
                # Extract tools used
                tools_used = extract_tools_from_steps(steps)
                
                # Extract response
                response_text = extract_response_text(data)
                
                # Display results
                print(f"\n   📊 Results:")
                print(f"      Steps: {len(steps)}")
                print(f"      Messages: {len(messages)}")
                
                if tools_used:
                    print(f"      🔧 Tools used: {', '.join(tools_used)}")
                else:
                    print(f"      🔧 Tools used: None")
                    # Show step types for debugging
                    step_types = [step.get('type', '') or step.get('step_type', '') for step in steps]
                    if step_types:
                        print(f"      📋 Step types: {', '.join(step_types)}")
                
                if response_text:
                    # Truncate long responses
                    display_text = response_text[:300] + "..." if len(response_text) > 300 else response_text
                    print(f"      💬 Response: {display_text}")
                else:
                    print(f"      ⚠️  No response text found")
                    # Try to show what we have
                    if steps:
                        print(f"      💡 Check Part 5 and Part 6 for detailed step information")
            else:
                print(f"   ⚠️  Could not retrieve turn data after retries")
        else:
            print("   ⚠️  Could not get turn_id from stream")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()

print("\n✅ Multiple query test complete!")
print("\n💡 Tip: For detailed tool usage and step information, check Part 5 and Part 6 above.")


---

## 🎓 Key Takeaways

**What we learned:**

1. **MCP (Model Context Protocol)** provides a standardized protocol for tool integration - one protocol, many tools!
2. **Agent Configuration** - Agents must be configured with toolgroups to access MCP tools
3. **Automatic Tool Usage** - Agents automatically decide when to use tools based on queries
4. **Turn-based Interaction** - Agents work through turns: create turn → stream → retrieve results
5. **Tool Execution Flow** - Agents call tools, receive results, and generate responses using real data

**The big picture:**
- **MCP** standardizes tool integration - build tools once, use with any agent
- **Agents** automatically use tools when needed - you don't need to explicitly call tools
- **Tool execution** is transparent - you can see what tools were called and their results
- **Real data** - Agents use actual data from MongoDB, not just training data

**For IT operations:**
- Build MCP tools for your monitoring systems, ticketing systems, databases
- Give agents access to your infrastructure through MCP
- Watch agents automatically use tools to answer questions
- Create production-ready agents that can actually manage your systems

**The workflow:**
1. Create agent with MCP toolgroups
2. Create session for conversation context
3. Query agent (it automatically uses tools!)
4. Retrieve turn results to see response and tool usage
5. Analyze steps to understand what happened

---

## 🚀 Next Steps

**Ready for more?** In **Notebook 05**, we'll explore:
- **Safety Shields** - Content moderation and safety checks (protect against harmful outputs)
- **Content Moderation** - Checking inputs and outputs for safety violations
- **Production-ready safety** - Building agents that are safe to deploy

**The fun part:** You'll learn how to protect your agents from harmful inputs and outputs - essential for production deployment!

---

**Ready?** Let's move to Notebook 05: Safety Shields! 🚀
